### 邮件形式发送

In [1]:
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# MySQL数据库连接信息
HOST = 'localhost'
USER = 'root'
PASSWORD = '123456'
DB = 'bigdata_etl'
PORT = 3306

# 监控字段变化的表格
TABLE_NAME = 'v2_job'

# 上次状态记录
last_state = {}

# 邮件提醒配置
MAIL_HOST = 'smtp.qq.com'  # QQ邮箱的SMTP服务器地址
MAIL_PORT = 587  # SMTP端口，587是常用端口
MAIL_USER = '2842351706@qq.com'  # 发件人QQ邮箱
MAIL_PASSWORD = 'tavcrllglnaidcij'  # 使用生成的授权码，而不是邮箱密码
TO_EMAIL = '1923557153@qq.com'  # 收件人的邮箱地址


# 邮件发送函数
def send_email(subject, body):
    msg = MIMEMultipart()
    msg['From'] = MAIL_USER
    msg['To'] = TO_EMAIL
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    # 连接SMTP服务器并发送邮件
    try:
        with smtplib.SMTP(MAIL_HOST, MAIL_PORT) as server:
            server.starttls()  # 启用TLS加密
            server.login(MAIL_USER, MAIL_PASSWORD)  # 使用授权码登录
            server.sendmail(MAIL_USER, TO_EMAIL, msg.as_string())  # 发送邮件
            print("邮件发送成功！")
    except smtplib.SMTPException as e:
        print(f"SMTP错误: {e}")
        return False
    except Exception as e:
        print(f"未知错误: {e}")
        return False


# 连接到MySQL数据库
def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


# 监控并报警
def monitor_job_status():
    global last_state
    connection = get_db_connection()
    try:
        with connection.cursor() as cursor:
            # 状态检测：
            query = (
                f" SELECT job_id,job_name, is_enable, execution_state, is_push FROM {TABLE_NAME} WHERE is_delete = 0 ")
            cursor.execute(query)
            results = cursor.fetchall()

            for row in results:
                job_id, job_name, is_enable, execution_state, is_push = row
                key = f'{job_id}'

                # 检查是否需要发出警报
                if key in last_state:
                    last_is_enable, last_execution_state, last_is_push = last_state[key]

                    if last_is_enable != is_enable:
                        status = '已被启用' if is_enable == 1 else '已禁用'
                        send_email(f"Job ID {job_id} ,{job_name} 状态变更", f"作业 {job_name}: {status}")

                    if last_is_push != is_push:
                        # status =' ' if last_is_push == 0 else
                        if is_push == 0:
                            send_email(f"Job ID {job_id} ,{job_name} 日志状态变更", f" {job_name}: '日志记录已被禁用'")

                    if last_execution_state != execution_state:
                        status = '执行中' if execution_state == 1 else (
                            '执行结束' if execution_state == 2 else '执行异常')
                        send_email(f"Job ID {job_id}, {job_name} 执行状态变更", f"作业 {job_name}: {status}")
                # 更新当前作业状态
                last_state[key] = (is_enable, execution_state, is_push)

    finally:
        connection.close()


# 设定监控间隔
def start_monitoring(interval=5):
    while True:
        monitor_job_status()
        time.sleep(interval)  # 每60秒监控一次


if __name__ == '__main__':
    start_monitoring()  # 启动监控，每60秒检查一次状态


邮件发送成功！
SMTP错误: (-1, b'\x00\x00\x00')


KeyboardInterrupt: 

### 机器人报警

In [5]:
import requests
import json
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# MySQL数据库连接信息
HOST = 'localhost'
USER = 'root'
PASSWORD = '123456'
DB = 'bigdata_etl'
PORT = 3306

# 监控表格
TABLE_NAME = 'v2_job'

# 上次状态记录
last_state = {}

WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'


def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            print(f"发送失败，状态码: {response.status_code}, 错误信息: {response.text}")
    except Exception as e:
        print(f"发生错误: {e}")


# 监控并报警
def monitor_job_status():
    global last_state
    con = get_db_connection()
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = (
                f" SELECT job_id,job_name, is_enable, execution_state, is_push FROM {TABLE_NAME} WHERE is_delete = 0 ")
            cursor.execute(query)
            results = cursor.fetchall()
            print(f'---results---:{results}')

            for row in results:
                job_id, job_name, is_enable, execution_state, is_push = row
                key = f'{job_id}'

                # 检查是否需要发出警报
                if key in last_state:
                    last_is_enable, last_execution_state, last_is_push = last_state[key]

                    if last_is_enable != is_enable:
                        status = '已启动' if is_enable == 1 else '已暂停'
                        print(f"ETL告警： JobId: {job_id} , {job_name}{status}")
                        # print(f"ETL告警： JobId: {job_id} , {job_name}{status}")
                        send_wechat_message(f"ETL告警： JobId: {job_id} , {job_name}{status}")

                    if last_is_push != is_push:
                        # status =' ' if last_is_push == 0 else
                        if is_push == 0:
                            print(f"ETL告警： JobId: {job_id} ,{job_name}的日志记录功能已被禁用")
                            send_wechat_message(f"ETL告警： JobId: {job_id} ,{job_name}的日志记录功能已被禁用")

                    if last_execution_state != execution_state:
                        status = '执行中' if execution_state == 1 else (
                            '执行结束' if execution_state == 2 else '执行异常')
                        print(f"ETL告警： JobId: {job_id}, {job_name}{status}")
                        send_wechat_message(f"ETL告警： JobId: {job_id}, {job_name}{status}")
                last_state[key] = (is_enable, execution_state, is_push)

    finally:
        con.close()


# 监控相关
def start_monitoring(interval=10):
    while True:
        monitor_job_status()
        time.sleep(interval)  # 每10秒监控一次


if __name__ == '__main__':
    start_monitoring()


---results---:((5, 't_wxwork', 1, 2, 1), (6, 't_wxwork_contact_code', 1, 2, 1), (7, 't_wxwork_customer', 1, 2, 1), (8, 't_wxwork_customer_groupchat', 1, 2, 1), (9, 't_wxwork_customer_groupchat_user', 1, 2, 0), (10, 't_wxwork_customer_tag', 1, 2, 1), (11, 't_wxwork_customer_user', 1, 2, 1), (12, 't_wxwork_external_contact_add_or_edit_msg_log', 0, 2, 1), (13, 't_wxwork_external_contact_delete_msg_log', 1, 2, 1), (14, 't_wxwork_map_info', 1, 2, 1), (15, 't_wxwork_group', 1, 2, 1), (16, 't_wxwork_tag', 1, 3, 1), (17, 't_wxwork_department', 1, 2, 1), (18, 't_wxwork_department_user', 1, 2, 1), (19, 't_wxwork_user', 1, 2, 1), (20, 't_wxwork_user_tag', 1, 2, 1), (21, 't_wxwork_user_visit_record', 1, 2, 1), (22, 't_wechat', 1, 2, 1), (23, 't_wechat_fans', 1, 2, 0), (24, 't_wechat_exttag', 1, 2, 1), (25, 't_wechat_fans_exttag', 1, 2, 1), (26, 't_wechat_tag', 1, 2, 1), (27, 't_wechat_qrcode', 1, 2, 1), (28, 'testopen', 0, 2, 1), (29, '葵花测试', 0, 2, 1), (30, 't_wechat_msg_log', 1, 2, 1), (31, 't_we

KeyboardInterrupt: 

In [7]:
import requests
import json
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# MySQL数据库连接信息
HOST = 'localhost'
USER = 'root'
PASSWORD = '123456'
DB = 'bigdata_etl'
PORT = 3306

# 监控表格
TABLE_NAME = 'v2_job'

# 上次状态记录
last_state = {}


WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'

def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            print(f"发送失败，状态码: {response.status_code}, 错误信息: {response.text}")
    except Exception as e:
        print(f"发生错误: {e}")


# 监控并报警
def monitor_job_status():
    global last_state
    con = get_db_connection()
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = (
                f" SELECT job_id,job_name, is_enable, execution_state, is_push FROM {TABLE_NAME} WHERE is_delete = 0 ")
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                job_id, job_name, is_enable, execution_state, is_push = row
                key = f'{job_id}'
                # 检查是否需要发出警报
                if key in last_state:
                    last_is_enable, last_execution_state, last_is_push = last_state[key]
                    if last_is_enable != is_enable and is_enable == 0:
                        status = '已暂停'
                        # print(f"ETL告警： JobId: {job_id} , {job_name}{status}")
                        send_wechat_message(f"ETL告警： JobId: {job_id} , {job_name}{status}")

                    if last_is_push != is_push and is_push == 1:
                        # print(f"ETL告警： JobId: {job_id} ,{job_name}的信息未推送")
                        send_wechat_message(f"ETL告警： JobId: {job_id} ,{job_name}的信息未推送")

                    if last_execution_state != execution_state and execution_state == 3:
                        status = '执行异常'
                        # print(f"ETL告警： JobId: {job_id}, {job_name}{status}")
                        send_wechat_message(f"ETL告警： JobId: {job_id}, {job_name}{status}")
                last_state[key] = (is_enable, execution_state, is_push)
    finally:
        con.close()


# 监控相关
def start_monitoring(interval=10):
    while True:
        monitor_job_status()
        time.sleep(interval)  # 每10秒监控一次


if __name__ == '__main__':
    start_monitoring()


ETL告警： JobId: 100, 葵花商城订单明细执行异常
ETL告警： JobId: 100 ,葵花商城订单明细的信息未推送
ETL告警： JobId: 65, tz_app_connect执行异常
ETL告警： JobId: 64 , tz_user_info已暂停
ETL告警： JobId: 64, tz_user_info执行异常


KeyboardInterrupt: 

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import time
import sys
import requests
import json
import pymysql
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

print
"标签分群检测预警"

# print "脚本位置：", sys.argv[0]
# print "任务参数：", sys.argv[1]
# print "分片序号：", sys.argv[2]
# print "分片总数：", sys.argv[3]

# MySQL数据库连接信息
# HOST = 'ba658a2d786448cbbff0a982bb4e2dc5in01.internal.cn-north-4.mysql.rds.myhuaweicloud.com'
HOST = '10.88.10.72'
# HOST = "127.0.0.1"
USER = 'root'
PASSWORD = 'YdaD6n9dBjF$l&D3'
DB = 'bigdata_general_tag'
PORT = 3306

# 上次状态记录
last_state = {}


# WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'


def get_db_connection():
    return pymysql.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            # print(f"发送失败，状态码: {response.status_code}, 错误信息: {response.text}")
            print
            "发送失败，状态码: %s, 错误信息: %s" % (response.status_code, response.text)
    except Exception as e:
        print
        "发生错误:", e


# 监控并报警
def monitor_tag_status():
    global last_state
    con = get_db_connection()
    msg="";
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = f'''
            SELECT 
                common_tag_id, 
                name, 
                common_tag_type,
                in_refresh, 
                TIMESTAMPDIFF(HOUR, 
                              (SELECT compute_start 
                               FROM common_tag 
                               WHERE compute_start < NOW() 
                                 AND compute_end < NOW() 
                                 AND common_tag_category_id IN (
                                     SELECT common_tag_category_id 
                                     FROM bigdata_general_tag.common_tag_category ctc 
                                     WHERE target_entity_api_name = 'xiaokuihua' 
                                       AND tenant_api_name = 't100000'
                                 )
                              ), NOW()) AS delay 
            FROM common_tag 
            WHERE (in_refresh = 1 OR in_refresh = 2)  
              AND compute_start < NOW()  
              AND compute_end < NOW()  
              AND common_tag_category_id IN (
                  SELECT common_tag_category_id 
                  FROM bigdata_general_tag.common_tag_category ctc 
                  WHERE target_entity_api_name = 'xiaokuihua' 
                    AND tenant_api_name = 't100000'
              )
            HAVING delay > 1;
            '''
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                common_tag_id, name, common_tag_type,in_refresh,delay= row
                name = name.encode('utf-8')
                # 检查是否需要发出警报
                if in_refresh == 1: # 1排队中 2计算中
                    if common_tag_type == 'tag': #标签
                        status = '标签排队时长已超1小时'
                    if common_tag_type == 'cluster': #分群
                        status = '分群排队时长已超1小时'
                    # print(f"ETL告警： JobId: {job_id} , {job_name}{status}")
                    msg = msg + "\n TAGID[%s] , %s %s" % (common_tag_id, name, status)
                if in_refresh == 2: # 1排队中 2计算中
                    if common_tag_type == 'tag': #标签
                        status = '标签计算时长已超1小时'
                    if common_tag_type == 'cluster': #分群
                        status = '分群计算时长已超1小时'
                    # print(f"ETL告警： JobId: {job_id} , {job_name}{status}")
                    msg = msg + "\n TAGID[%s] , %s %s" % (common_tag_id, name, status)

            if msg:
                msg = "标签分群告警:"+msg
                # send_wechat_message(msg)
                print(f"{msg}")
    finally:
        con.close()

# 监控相关
def start_monitoring(interval=10):
    while True:
        monitor_tag_status()
        time.sleep(interval)  # 每10秒监控一次


if __name__ == '__main__':
    start_monitoring()

# monitor_tag_status();

print
"Task Finish!"
exit(0)

In [9]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import time
import sys
import requests
import json
import psycopg2
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

print
"MA监控"

# print "脚本位置：", sys.argv[0]
# print "任务参数：", sys.argv[1]
# print "分片序号：", sys.argv[2]
# print "分片总数：", sys.argv[3]

# MySQL数据库连接信息
HOST = '10.88.10.73'
USER = 'postgres'
PASSWORD = 'R4BuypN9rgeUt7qk'
DB = 'bigdata_flow'
PORT = 5432

# 上次状态记录
last_state = {}


# WEBHOOK_URL = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=372def7c-1657-4ee3-bf69-0d3b44581d14'


def get_db_connection():
    return psycopg2.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        db=DB,
        port=PORT,
        charset='utf8mb4'
    )


def send_wechat_message(content):
    message = {
        "msgtype": "text",  # 消息类型
        "at": {
            "isAtAll": "true"
        },
        "text": {
            "content": content  # 发送的消息内容
        }
    }

    try:
        response = requests.post(WEBHOOK_URL, data=json.dumps(message), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print("消息发送成功")
        else:
            # print(f"发送失败，状态码: {response.status_code}, 错误信息: {response.text}")
            print
            "发送失败，状态码: %s, 错误信息: %s" % (response.status_code, response.text)
    except Exception as e:
        print
        "发生错误:", e


# 监控并报警
def monitor_ma_status():
    global last_state
    con = get_db_connection()
    msg="";
    try:
        with con.cursor() as cursor:
            # 状态检测：
            query = f'''
            select flow_id, flow_name
            from (select fi.flow_id,
                         fi.flow_name,
                         fi.flow_status,
                         SUM(CASE "node_key" WHEN 'start' THEN total ELSE 0 END) as stt,
                         SUM(CASE "node_key" WHEN 'end' THEN total ELSE 0 END)   as ed
                  from public.flow_info fi
                           left join nodedata.flow_data_log_agg fla on fi.flow_id = fla.flow_id
                  where fi.flow_id in (select flow_id
                                       from flow_info fi
                                       where flow_status = 3)
                  group by fi.flow_id, fi.flow_name, fi.flow_status) a
            where stt <> ed;
            '''
            cursor.execute(query)
            results = cursor.fetchall()
            for row in results:
                flow_id, flow_name= row
                flow_name = flow_name.encode('utf-8')
                status = "执行进度未达到100%，存在异常"
                # 检查是否需要发出警报
                msg = msg + "\n MAID[%s] , %s执行已结束，%s" % (flow_id, flow_name, status)

            if msg:
                msg = "MA监控告警:" + msg
                # send_wechat_message(msg)
                print(f"{msg}")
    finally:
        con.close()

# 监控相关
# def start_monitoring(interval=0):
#     while True:
#         monitor_ma_status()
#         time.sleep(interval)  # 每10秒监控一次
# 
# 
# if __name__ == '__main__':
#     start_monitoring()

monitor_ma_status();

print
"Task Finish!"
exit(0)


ProgrammingError: invalid dsn: invalid connection option "db"
